In [1]:
#singleton pattern object builder
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("241206_03_DataFrameAPI").getOrCreate()
spark

24/12/09 10:29:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.format("csv").load("data/2015-summary.csv", interSchema=True, header=True)
df

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: string]

In [3]:
print(df.schema)
print(df.printSchema())

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,StringType,true)))
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)

None


## Row 클래스, 단일 레코드(행)을 나타내는 객체

Row(DES_COUNTRY_NAME = 'United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [4]:
df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count='15'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count='1'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count='344'),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count='15'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count='62')]

In [5]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [6]:
#spark DataTable
df.select("DEST_COUNTRY_NAME").show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [7]:
df.count()

256

In [8]:
df.select("DEST_COUNTRY_NAME").count()

256

In [9]:
#중복 확인
df_dup = df.select("DEST_COUNTRY_NAME").dropDuplicates()
df_dup.show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [10]:
df_dup = df.select("DEST_COUNTRY_NAME").dropDuplicates().cache()
df_dup.show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [11]:
df_dup.distinct().count()

132

In [12]:
df.sort("DEST_COUNTRY_NAME").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Algeria|      United States|    4|
|              Angola|      United States|   15|
|            Anguilla|      United States|   41|
| Antigua and Barbuda|      United States|  126|
|           Argentina|      United States|  180|
|               Aruba|      United States|  346|
|           Australia|      United States|  329|
|             Austria|      United States|   62|
|          Azerbaijan|      United States|   21|
|             Bahrain|      United States|   19|
|            Barbados|      United States|  154|
|             Belgium|      United States|  259|
|              Belize|      United States|  188|
|             Bermuda|      United States|  183|
|             Bolivia|      United States|   30|
|Bonaire, Sint Eus...|      United States|   58|
|              Brazil|      United States|  853|
|British Virgin Is..

In [13]:
from pyspark.sql.functions import *

#새로 컬럼 추가
df3 = df.withColumn('withInCountry',expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))
df3.show(5)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withInCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
|    United States|              India|   62|        false|
+-----------------+-------------------+-----+-------------+
only showing top 5 rows



In [14]:
#SQL 구문 - CASE WHEN > 수치형 변수 > 명목형 변수로 변환

df4 = df.withColumn('Category',expr("CASE WHEN count <10 THEN 'under' WHEN count >=10 THEN 'upper' END"))
df4.show(5)

+-----------------+-------------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Category|
+-----------------+-------------------+-----+--------+
|    United States|            Romania|   15|   upper|
|    United States|            Croatia|    1|   under|
|    United States|            Ireland|  344|   upper|
|            Egypt|      United States|   15|   upper|
|    United States|              India|   62|   upper|
+-----------------+-------------------+-----+--------+
only showing top 5 rows



In [15]:
df5 = df4.withColumn('withInColumn',expr("ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME"))
df5.show(5)

+-----------------+-------------------+-----+--------+------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Category|withInColumn|
+-----------------+-------------------+-----+--------+------------+
|    United States|            Romania|   15|   upper|       false|
|    United States|            Croatia|    1|   under|       false|
|    United States|            Ireland|  344|   upper|       false|
|            Egypt|      United States|   15|   upper|       false|
|    United States|              India|   62|   upper|       false|
+-----------------+-------------------+-----+--------+------------+
only showing top 5 rows



In [16]:
df5.groupBy("withInColumn").count().show()

+------------+-----+
|withInColumn|count|
+------------+-----+
|        true|    1|
|       false|  255|
+------------+-----+



## Projection 과 Filter
```
SELECT a,b,c # projection > column > Transformation select('col name')
FROM Table A 
WHERE a>10 #filter > Row > Transformation where('condition')
```

In [17]:
#filter 
df6 = df5.where('count<5')
df6.show()
df6.count()

+--------------------+-------------------+-----+--------+------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Category|withInColumn|
+--------------------+-------------------+-----+--------+------------+
|       United States|            Croatia|    1|   under|       false|
|       United States|          Singapore|    1|   under|       false|
|             Moldova|      United States|    1|   under|       false|
|               Malta|      United States|    1|   under|       false|
|             Algeria|      United States|    4|   under|       false|
|       United States|          Gibraltar|    1|   under|       false|
|Saint Vincent and...|      United States|    1|   under|       false|
|            Suriname|      United States|    1|   under|       false|
|       United States|             Cyprus|    1|   under|       false|
|       United States|           Malaysia|    3|   under|       false|
|            Thailand|      United States|    3|   under|       false|
|     

46

In [18]:
df6 = df5.where('count<5').where("ORIGIN_COUNTRY_NAME != 'United States'")
df6.show()
df6.count()

+-----------------+-------------------+-----+--------+------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Category|withInColumn|
+-----------------+-------------------+-----+--------+------------+
|    United States|            Croatia|    1|   under|       false|
|    United States|          Singapore|    1|   under|       false|
|    United States|          Gibraltar|    1|   under|       false|
|    United States|             Cyprus|    1|   under|       false|
|    United States|           Malaysia|    3|   under|       false|
|    United States|            Vietnam|    2|   under|       false|
|    United States|            Estonia|    1|   under|       false|
|    United States|            Hungary|    3|   under|       false|
|    United States|           Thailand|    4|   under|       false|
|    United States|            Liberia|    2|   under|       false|
|    United States|              Malta|    2|   under|       false|
|    United States|          Lithuania|    1|   

20

In [19]:
#프로젝션, 필터링 연습 
df5 = df5.withColumn("count", col("count").cast("int")) # count 가 string 이네..? int 로 변경-- cast()
# df5.show(10)

#국내 여행이 아니면서 가장 횟수가 많은 ORIGIN_COUNTRY_NAME top 10을  추출해보세요
top10_trip = df5.where("ORIGIN_COUNTRY_NAME != DEST_COUNTRY_NAME") \
                .groupBy("ORIGIN_COUNTRY_NAME") \
                .sum("count") \
                .sort("sum(count)", ascending=False)
top10_trip.show(10)

+-------------------+----------+
|ORIGIN_COUNTRY_NAME|sum(count)|
+-------------------+----------+
|      United States|     41964|
|             Canada|      8483|
|             Mexico|      7187|
|     United Kingdom|      1970|
|              Japan|      1496|
| Dominican Republic|      1420|
|            Germany|      1336|
|        The Bahamas|       986|
|             France|       952|
|              China|       920|
+-------------------+----------+
only showing top 10 rows



In [20]:
#국내 여행이 아니면서 가장 횟수가 적은 ORIGIN_COUNTRY_NAME top 10을  추출해보세요
top10_trip_least = df5.where("ORIGIN_COUNTRY_NAME != DEST_COUNTRY_NAME")\
                .groupBy("ORIGIN_COUNTRY_NAME").count()\
                .sort("count", ascending=True)
top10_trip_least.show(10)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Senegal|    1|
|             Sweden|    1|
|           Paraguay|    1|
|             Russia|    1|
|           Anguilla|    1|
|           Kiribati|    1|
|             Guyana|    1|
|          Singapore|    1|
|           Malaysia|    1|
|        Philippines|    1|
+-------------------+-----+
only showing top 10 rows



In [21]:
#도착국가별 count 총합이 가장 많은 top10을 추출해 보세요
df7 = df5.groupBy("DEST_COUNTRY_NAME").sum("count").sort('sum(count)',ascending = False)
df7.show(10)

+------------------+----------+
| DEST_COUNTRY_NAME|sum(count)|
+------------------+----------+
|     United States|    411352|
|            Canada|      8399|
|            Mexico|      7140|
|    United Kingdom|      2025|
|             Japan|      1548|
|           Germany|      1468|
|Dominican Republic|      1353|
|       South Korea|      1048|
|       The Bahamas|       955|
|            France|       935|
+------------------+----------+
only showing top 10 rows



In [22]:
#카테고리별 분석
df6 = df5.withColumn("count", col("count").cast("int"))
df6.groupby("Category").sum("count").show()

+--------+----------+
|Category|sum(count)|
+--------+----------+
|   under|        91|
|   upper|    453225|
+--------+----------+



In [23]:
#비행 횟수가 1000 이상 필터링
df6.filter(df6['count']>=1000).show()

+------------------+-------------------+------+--------+------------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|Category|withInColumn|
+------------------+-------------------+------+--------+------------+
|            Mexico|      United States|  7140|   upper|       false|
|     United States| Dominican Republic|  1420|   upper|       false|
|     United States|      United States|370002|   upper|        true|
|           Germany|      United States|  1468|   upper|       false|
|            Canada|      United States|  8399|   upper|       false|
|Dominican Republic|      United States|  1353|   upper|       false|
|             Japan|      United States|  1548|   upper|       false|
|     United States|            Germany|  1336|   upper|       false|
|     United States|             Mexico|  7187|   upper|       false|
|    United Kingdom|      United States|  2025|   upper|       false|
|     United States|              Japan|  1496|   upper|       false|
|     United States|

In [24]:
#상위 10 목적지 국가 
df6.groupBy("DEST_COUNTRY_NAME").sum("count").sort("sum(count)", ascending = False).show(10)

+------------------+----------+
| DEST_COUNTRY_NAME|sum(count)|
+------------------+----------+
|     United States|    411352|
|            Canada|      8399|
|            Mexico|      7140|
|    United Kingdom|      2025|
|             Japan|      1548|
|           Germany|      1468|
|Dominican Republic|      1353|
|       South Korea|      1048|
|       The Bahamas|       955|
|            France|       935|
+------------------+----------+
only showing top 10 rows



In [25]:
#spark SQL
df.createOrReplaceTempView("mobility_data")
spark.sql("select * from mobility_data").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [27]:
spark.stop()

# 집계 함수

In [171]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SecondSparkSessionApp").getOrCreate()
spark

In [172]:
df = spark.read.format("csv")\
    .option("header",'true')\
    .option('inferSchema', 'true')\
    .load("data/emp.csv")

In [173]:
df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [174]:
df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
| 9292|  JACK|

In [175]:
df.select('ename','deptno').show()

+------+------+
| ename|deptno|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [176]:
df.select('ename','deptno').where('deptno=20').show()

+-----+------+
|ename|deptno|
+-----+------+
|SMITH|    20|
|JONES|    20|
|SCOTT|    20|
|ADAMS|    20|
| FORD|    20|
+-----+------+



In [177]:
# 카운트 집계
#null 값이 제외
from pyspark.sql.functions import count, countDistinct, approx_count_distinct
df.select(count('job')).show()

+----------+
|count(job)|
+----------+
|        15|
+----------+



In [178]:
#null 값이 포함
df.selectExpr('count(comm)').show()

+-----------+
|count(comm)|
+-----------+
|          4|
+-----------+



In [179]:
df.select('job').distinct().show()

+---------+
|      job|
+---------+
|  ANALYST|
| SALESMAN|
|    CLERK|
|  MANAGER|
|PRESIDENT|
+---------+



In [82]:
df.select('job').distinct().count() #정확도 

5

In [83]:
df.select(countDistinct('job')).show() #근사치

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [84]:
df.select(approx_count_distinct('job', 0.1)).show() #지정한 오차가 있어도 성능면에서 유리한 연산 

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [122]:
from pyspark.sql.functions import *

In [123]:
df.select(first('ename'),last('ename')).show()

+------------+-----------+
|first(ename)|last(ename)|
+------------+-----------+
|       SMITH|       JACK|
+------------+-----------+



In [124]:
# min, max
df.select(min('mgr'), max('mgr')).show()

+--------+--------+
|min(mgr)|max(mgr)|
+--------+--------+
|    7566|    7902|
+--------+--------+



In [125]:
df.select(min('hiredate'), max('hiredate')).show()

+-------------+-------------+
|min(hiredate)|max(hiredate)|
+-------------+-------------+
|   1980-12-17|   1987-05-23|
+-------------+-------------+



In [126]:
df.select(count("empno"), count("*"), max("ename"), min("ename")).show()

+------------+--------+----------+----------+
|count(empno)|count(1)|max(ename)|min(ename)|
+------------+--------+----------+----------+
|          15|      15|      WARD|     ADAMS|
+------------+--------+----------+----------+



In [127]:
df.select(sum("sal")).show() #sal 컬럽의 총합 

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [128]:
# sal 컬럼값의 중복을 제거하고 합산 

df.selectExpr('sum(distinct sal)').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [141]:
#alias 
dfs = df.select(
                count('sal').alias('total_tx'),
                sum('sal').alias('total_salary'),
                avg('sal').alias('avg_salary'),
                expr('mean(sal)').alias('mean_salary')
)
dfs.show()

+--------+------------+------------------+------------------+
|total_tx|total_salary|        avg_salary|       mean_salary|
+--------+------------+------------------+------------------+
|      15|       32225|2148.3333333333335|2148.3333333333335|
+--------+------------+------------------+------------------+



In [146]:
#round (data, 자릿수)
dfs = df.select(
                count('sal').alias('total_tx'),
                sum('sal').alias('total_salary'),
                round(avg('sal'),2).alias('avg_salary'),
                round(expr('mean(sal)'),2).alias('mean_salary'),
        )
dfs.show()

+--------+------------+----------+-----------+
|total_tx|total_salary|avg_salary|mean_salary|
+--------+------------+----------+-----------+
|      15|       32225|   2148.33|    2148.33|
+--------+------------+----------+-----------+



# 그룹화

In [142]:
df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [150]:
#agg() 집계 함수 적용

dfs = df.groupBy('job').agg(expr('avg(sal) as SAL_AVG'))
dfs.show()

+---------+------------------+
|      job|           SAL_AVG|
+---------+------------------+
|  ANALYST|            3000.0|
| SALESMAN|            1400.0|
|    CLERK|            1470.0|
|  MANAGER|2758.3333333333335|
|PRESIDENT|            5000.0|
+---------+------------------+



In [166]:
#표준편차 구하기 
# 1. sql.function stddev
# 2.sql expression

#1번째 방법
dfs=df.groupBy('job').agg(round(stddev_pop('sal'),2)).alias('SAL_STDDEV')
dfs.show()

+---------+-------------------------+
|      job|round(stddev_pop(sal), 2)|
+---------+-------------------------+
|  ANALYST|                      0.0|
| SALESMAN|                   154.11|
|    CLERK|                   880.68|
|  MANAGER|                   223.92|
|PRESIDENT|                      0.0|
+---------+-------------------------+



In [167]:
#2번째 방법
dfs = df.groupBy('job').agg(round(expr('stddev_pop(sal) as SAL_STDDEV'),2))
dfs.show()

+---------+-----------------------------------------+
|      job|round(stddev_pop(sal) AS `SAL_STDDEV`, 2)|
+---------+-----------------------------------------+
|  ANALYST|                                      0.0|
| SALESMAN|                                   154.11|
|    CLERK|                                   880.68|
|  MANAGER|                                   223.92|
|PRESIDENT|                                      0.0|
+---------+-----------------------------------------+



# 윈도우 함수 (Window Function)

```
순위, 정렬 - rank, row_number, dense_rank
누계 - sum, avg, max, min + over()
이동평균, 이동합계 - over + rowsBetween, rangeBetween
시차, 선행 = lag, lead

ex) 세선 구간내 분석, 특정시간 동안 일어난 활동 그룹화

```

1. partitionBy() > 소그룹으로 나눈다. 
2. orderBy() > 소그룹 내 정렬
3. rowBetween(), rangeBetween()
4. over()

In [223]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

# 순위를 부여하려고 하는 데이터의 범위 > 윈도우 명세 설정
windowspec = Window.orderBy(desc('sal'))
windowspec

In [224]:
# 순위 객체
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [225]:
#데이터 프레임에 컬럼으로 추가 > 액션
df.withColumn("salary_rank", salAllRank).show()

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|null|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|         10|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|         11|
| 7654|MARTIN| SALES

24/12/09 13:59:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [226]:
df.select('empno', salAllRank.alias("salary_rank")).show()

+-----+-----------+
|empno|salary_rank|
+-----+-----------+
| 7839|          1|
| 9292|          2|
| 7788|          3|
| 7902|          3|
| 7566|          5|
| 7698|          6|
| 7782|          7|
| 7499|          8|
| 7844|          9|
| 7934|         10|
| 7521|         11|
| 7654|         11|
| 7876|         13|
| 7900|         14|
| 7369|         15|
+-----+-----------+



24/12/09 13:59:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [227]:
#집업별 급여 순위
#windowspec = Window.orderBy(desc('sal'))
windowspec = Window.partitionBy('Job').orderBy(desc('sal'))
salJobrank = rank().over(windowspec)

In [228]:
df.select(
    'job','ename','sal',
    salJobrank.alias('salJobRank')
).show()

+---------+------+----+----------+
|      job| ename| sal|salJobRank|
+---------+------+----+----------+
|  ANALYST| SCOTT|3000|         1|
|  ANALYST|  FORD|3000|         1|
| SALESMAN| ALLEN|1600|         1|
| SALESMAN|TURNER|1500|         2|
| SALESMAN|  WARD|1250|         3|
| SALESMAN|MARTIN|1250|         3|
|    CLERK|  JACK|3200|         1|
|    CLERK|MILLER|1300|         2|
|    CLERK| ADAMS|1100|         3|
|    CLERK| JAMES| 950|         4|
|    CLERK| SMITH| 800|         5|
|  MANAGER| JONES|2975|         1|
|  MANAGER| BLAKE|2850|         2|
|  MANAGER| CLARK|2450|         3|
|PRESIDENT|  KING|5000|         1|
+---------+------+----+----------+



In [229]:
#부서별 급여 순위
#windowspec = Window.orderBy(desc('sal'))
windowspec2 = Window.partitionBy('deptno').orderBy(desc('sal'))
salDeptrank = rank().over(windowspec2)

In [230]:
df.select(
    'deptno','ename','sal',
    salJobrank.alias('salDeptRank')
).orderBy('deptno', 'salDeptRank').show()

+------+------+----+-----------+
|deptno| ename| sal|salDeptRank|
+------+------+----+-----------+
|    10|  KING|5000|          1|
|    10|MILLER|1300|          2|
|    10| CLARK|2450|          3|
|    20| SCOTT|3000|          1|
|    20|  FORD|3000|          1|
|    20| JONES|2975|          1|
|    20| ADAMS|1100|          3|
|    20| SMITH| 800|          5|
|    30| ALLEN|1600|          1|
|    30|TURNER|1500|          2|
|    30| BLAKE|2850|          2|
|    30|  WARD|1250|          3|
|    30|MARTIN|1250|          3|
|    30| JAMES| 950|          4|
|    70|  JACK|3200|          1|
+------+------+----+-----------+



In [234]:
#다른 답

#rank()
dept_window_spec = Window.partitionBy('deptno').orderBy(desc('sal'))
new_df = df.withColumn('dept_salary_rank', rank().over(dept_window_spec))
new_df.select(
    'ename', 'deptno','sal','dept_salary_rank'
).show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               1|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
|  JACK|    70|3200|               1|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               4|
| JAMES|    30| 950|               6|
+------+------+----+----------------+



In [235]:
#row_number
from pyspark.sql.functions import row_number
new_df = df.withColumn('dept_salary_rank', row_number().over(dept_window_spec))
new_df.select(
    'ename', 'deptno','sal','dept_salary_rank'
).show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               2|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
|  JACK|    70|3200|               1|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               5|
| JAMES|    30| 950|               6|
+------+------+----+----------------+



In [238]:
#dense_rank()
from pyspark.sql.functions import dense_rank
new_df = df.withColumn('dept_salary_rank', dense_rank().over(dept_window_spec))
new_df.select(
    'ename', 'deptno','sal','dept_salary_rank'
).show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               1|
| JONES|    20|2975|               2|
| ADAMS|    20|1100|               3|
| SMITH|    20| 800|               4|
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
|  JACK|    70|3200|               1|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               4|
| JAMES|    30| 950|               5|
+------+------+----+----------------+



In [242]:
#누적 급여 계산 

sum_window_spec = Window.partitionBy('deptno').orderBy('empno')

new_df = df.withColumn('sum_salary', sum('sal').over(sum_window_spec))
new_df.select(
    'ename', 'deptno','sal','sum_salary'
).show()

+------+------+----+----------+
| ename|deptno| sal|sum_salary|
+------+------+----+----------+
| SMITH|    20| 800|       800|
| JONES|    20|2975|      3775|
| SCOTT|    20|3000|      6775|
| ADAMS|    20|1100|      7875|
|  FORD|    20|3000|     10875|
| CLARK|    10|2450|      2450|
|  KING|    10|5000|      7450|
|MILLER|    10|1300|      8750|
|  JACK|    70|3200|      3200|
| ALLEN|    30|1600|      1600|
|  WARD|    30|1250|      2850|
|MARTIN|    30|1250|      4100|
| BLAKE|    30|2850|      6950|
|TURNER|    30|1500|      8450|
| JAMES|    30| 950|      9400|
+------+------+----+----------+



In [251]:
# 부서별 평균 급여 계산 

avg_window_spec = Window.partitionBy('deptno')

new_df = df.withColumn('avg_salary', round(avg('sal').over(avg_window_spec),2))
new_df.select(
    'ename', 'deptno','sal','avg_salary'
).show()

+------+------+----+----------+
| ename|deptno| sal|avg_salary|
+------+------+----+----------+
| SMITH|    20| 800|    2175.0|
| JONES|    20|2975|    2175.0|
| SCOTT|    20|3000|    2175.0|
| ADAMS|    20|1100|    2175.0|
|  FORD|    20|3000|    2175.0|
| CLARK|    10|2450|   2916.67|
|  KING|    10|5000|   2916.67|
|MILLER|    10|1300|   2916.67|
|  JACK|    70|3200|    3200.0|
| ALLEN|    30|1600|   1566.67|
|  WARD|    30|1250|   1566.67|
|MARTIN|    30|1250|   1566.67|
| BLAKE|    30|2850|   1566.67|
|TURNER|    30|1500|   1566.67|
| JAMES|    30| 950|   1566.67|
+------+------+----+----------+



In [ ]:
#SQL SELECT 문

SELECT
ename,deptno, sal, #projection
avg('sal').over(partition by deptno) as dept_avg_salary
from emp; #filter

In [256]:
df.createOrReplaceTempView("emp")  # 'emp' 테이블 생성

query = """
SELECT 
    ename, 
    deptno, 
    sal, 
    ROUND(AVG(sal) OVER (PARTITION BY deptno), 2) AS dept_avg_salary
FROM emp
"""

result_df = spark.sql(query)
result_df.show()

+------+------+----+---------------+
| ename|deptno| sal|dept_avg_salary|
+------+------+----+---------------+
| SMITH|    20| 800|         2175.0|
| JONES|    20|2975|         2175.0|
| SCOTT|    20|3000|         2175.0|
| ADAMS|    20|1100|         2175.0|
|  FORD|    20|3000|         2175.0|
| CLARK|    10|2450|        2916.67|
|  KING|    10|5000|        2916.67|
|MILLER|    10|1300|        2916.67|
|  JACK|    70|3200|         3200.0|
| ALLEN|    30|1600|        1566.67|
|  WARD|    30|1250|        1566.67|
|MARTIN|    30|1250|        1566.67|
| BLAKE|    30|2850|        1566.67|
|TURNER|    30|1500|        1566.67|
| JAMES|    30| 950|        1566.67|
+------+------+----+---------------+



In [263]:
# lead, lag 이전 급여, 이후 급여
from pyspark.sql.functions import lag,lead

row_window_spec = Window.partitionBy('deptno').orderBy('empno')
lead_lag_new_avg_df = df.withColumn('prev_salary', lag('sal').over(row_window_spec))\
                .withColumn('next_salary', lead('sal').over(row_window_spec))

lead_lag_new_avg_df.select('ename','deptno','sal','prev_salary','next_salary').show()

+------+------+----+-----------+-----------+
| ename|deptno| sal|prev_salary|next_salary|
+------+------+----+-----------+-----------+
| SMITH|    20| 800|       null|       2975|
| JONES|    20|2975|        800|       3000|
| SCOTT|    20|3000|       2975|       1100|
| ADAMS|    20|1100|       3000|       3000|
|  FORD|    20|3000|       1100|       null|
| CLARK|    10|2450|       null|       5000|
|  KING|    10|5000|       2450|       1300|
|MILLER|    10|1300|       5000|       null|
|  JACK|    70|3200|       null|       null|
| ALLEN|    30|1600|       null|       1250|
|  WARD|    30|1250|       1600|       1250|
|MARTIN|    30|1250|       1250|       2850|
| BLAKE|    30|2850|       1250|       1500|
|TURNER|    30|1500|       2850|        950|
| JAMES|    30| 950|       1500|       null|
+------+------+----+-----------+-----------+



In [272]:
#SQL SELECT 문
df.createOrReplaceTempView("emp") 

query2 = '''
SELECT ename, deptno, sal,
LAG(sal) OVER(PARTITION  BY deptno order by empno) as prev_salary,
LEAD(sal) OVER (PARTITION BY deptno order by empno) as next_salary
FROM emp;
'''
result_df2 = spark.sql(query2)
result_df2.show()

+------+------+----+-----------+-----------+
| ename|deptno| sal|prev_salary|next_salary|
+------+------+----+-----------+-----------+
| SMITH|    20| 800|       null|       2975|
| JONES|    20|2975|        800|       3000|
| SCOTT|    20|3000|       2975|       1100|
| ADAMS|    20|1100|       3000|       3000|
|  FORD|    20|3000|       1100|       null|
| CLARK|    10|2450|       null|       5000|
|  KING|    10|5000|       2450|       1300|
|MILLER|    10|1300|       5000|       null|
|  JACK|    70|3200|       null|       null|
| ALLEN|    30|1600|       null|       1250|
|  WARD|    30|1250|       1600|       1250|
|MARTIN|    30|1250|       1250|       2850|
| BLAKE|    30|2850|       1250|       1500|
|TURNER|    30|1500|       2850|        950|
| JAMES|    30| 950|       1500|       null|
+------+------+----+-----------+-----------+



# rollup, cube

```
over(), groupby()
rollup: 계층적집계, 부분합(subtotal), 총합(grandtotal)
cube: 모든 값으로 부분합, 결합 가능합 모든 값의 부분합을 구한다
```

In [273]:
df

DataFrame[empno: int, ename: string, job: string, mgr: int, hiredate: string, sal: int, comm: int, deptno: int]

In [278]:
#그룹화 > 소계

df.groupBy('deptno','job')\
    .agg(count('*'), sum('sal'))\
    .orderBy('deptno','job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [288]:
#rollup
df.rollup('deptno','job')\
    .agg(count('*'), sum('sal'))\
    .orderBy('deptno','job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [289]:
#cube
df.cube('deptno','job')\
    .agg(count('*'), sum('sal'))\
    .orderBy('deptno','job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|  null|  ANALYST|       2|    6000|
|  null|    CLERK|       5|    7350|
|  null|  MANAGER|       3|    8275|
|  null|PRESIDENT|       1|    5000|
|  null| SALESMAN|       4|    5600|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [320]:
#롤업 -> 대,중분류 최대/최소값

df.rollup('deptno','job')\
    .agg(count('*').alias('count'),
         min('sal').alias('min_salary'), 
         max('sal').alias('max_salary'))\
    .orderBy('deptno','job').show()

+------+---------+-----+----------+----------+
|deptno|      job|count|min_salary|max_salary|
+------+---------+-----+----------+----------+
|  null|     null|   15|       800|      5000|
|    10|     null|    3|      1300|      5000|
|    10|    CLERK|    1|      1300|      1300|
|    10|  MANAGER|    1|      2450|      2450|
|    10|PRESIDENT|    1|      5000|      5000|
|    20|     null|    5|       800|      3000|
|    20|  ANALYST|    2|      3000|      3000|
|    20|    CLERK|    2|       800|      1100|
|    20|  MANAGER|    1|      2975|      2975|
|    30|     null|    6|       950|      2850|
|    30|    CLERK|    1|       950|       950|
|    30|  MANAGER|    1|      2850|      2850|
|    30| SALESMAN|    4|      1250|      1600|
|    70|     null|    1|      3200|      3200|
|    70|    CLERK|    1|      3200|      3200|
+------+---------+-----+----------+----------+



In [291]:
#SQL SELECT 문
df.createOrReplaceTempView("emp") 

query = '''
SELECT deptno,job,
max(sal) as max_salary,
min(sal) as min_salary
FROM emp
group by rollup(deptno, job)
order by deptno, job;
'''
result_df = spark.sql(query)
result_df.show()

+------+---------+----------+----------+
|deptno|      job|max_salary|min_salary|
+------+---------+----------+----------+
|  null|     null|      5000|       800|
|    10|     null|      5000|      1300|
|    10|    CLERK|      1300|      1300|
|    10|  MANAGER|      2450|      2450|
|    10|PRESIDENT|      5000|      5000|
|    20|     null|      3000|       800|
|    20|  ANALYST|      3000|      3000|
|    20|    CLERK|      1100|       800|
|    20|  MANAGER|      2975|      2975|
|    30|     null|      2850|       950|
|    30|    CLERK|       950|       950|
|    30|  MANAGER|      2850|      2850|
|    30| SALESMAN|      1600|      1250|
|    70|     null|      3200|      3200|
|    70|    CLERK|      3200|      3200|
+------+---------+----------+----------+



In [319]:
# job 별 평균 급여 job, avg_salary, total_salary
df.rollup('job')\
    .agg(round(avg('sal'),2).alias('avg_salary'),
         sum('sal').alias('total_salary')).orderBy('job').show()

+---------+----------+------------+
|      job|avg_salary|total_salary|
+---------+----------+------------+
|     null|   2148.33|       32225|
|  ANALYST|    3000.0|        6000|
|    CLERK|    1470.0|        7350|
|  MANAGER|   2758.33|        8275|
|PRESIDENT|    5000.0|        5000|
| SALESMAN|    1400.0|        5600|
+---------+----------+------------+



In [317]:
#SQL SELECT 문
#df.createOrReplaceTempView("emp") 

query = '''
SELECT job,
round(avg(sal),2) as avg_salary,
sum(sal) as total_salary
FROM emp
group by rollup(job)
order by job;
'''
result_df = spark.sql(query)
result_df.show()

+---------+----------+------------+
|      job|avg_salary|total_salary|
+---------+----------+------------+
|     null|   2148.33|       32225|
|  ANALYST|    3000.0|        6000|
|    CLERK|    1470.0|        7350|
|  MANAGER|   2758.33|        8275|
|PRESIDENT|    5000.0|        5000|
| SALESMAN|    1400.0|        5600|
+---------+----------+------------+



In [318]:
# deptno, job 평균급여, 최대급여를 모두 소계를 냅니다. - cube

df.cube('deptno','job')\
    .agg(round(avg('sal'),2).alias('avg_salary'),
    max('sal').alias('max_salary'))\
    .orderBy('deptno','job').show()

+------+---------+----------+----------+
|deptno|      job|avg_salary|max_salary|
+------+---------+----------+----------+
|  null|     null|   2148.33|      5000|
|  null|  ANALYST|    3000.0|      3000|
|  null|    CLERK|    1470.0|      3200|
|  null|  MANAGER|   2758.33|      2975|
|  null|PRESIDENT|    5000.0|      5000|
|  null| SALESMAN|    1400.0|      1600|
|    10|     null|   2916.67|      5000|
|    10|    CLERK|    1300.0|      1300|
|    10|  MANAGER|    2450.0|      2450|
|    10|PRESIDENT|    5000.0|      5000|
|    20|     null|    2175.0|      3000|
|    20|  ANALYST|    3000.0|      3000|
|    20|    CLERK|     950.0|      1100|
|    20|  MANAGER|    2975.0|      2975|
|    30|     null|   1566.67|      2850|
|    30|    CLERK|     950.0|       950|
|    30|  MANAGER|    2850.0|      2850|
|    30| SALESMAN|    1400.0|      1600|
|    70|     null|    3200.0|      3200|
|    70|    CLERK|    3200.0|      3200|
+------+---------+----------+----------+



In [315]:
#SQL SELECT 문
#df.createOrReplaceTempView("emp") 

query = '''
SELECT deptno, job,
round(avg(sal),2) as avg_salary,
max(sal) as max_salary
FROM emp
group by cube(deptno, job)
order by deptno, job;
'''
result_df = spark.sql(query)
result_df.show()

+------+---------+----------+----------+
|deptno|      job|avg_salary|max_salary|
+------+---------+----------+----------+
|  null|     null|   2148.33|      5000|
|  null|  ANALYST|    3000.0|      3000|
|  null|    CLERK|    1470.0|      3200|
|  null|  MANAGER|   2758.33|      2975|
|  null|PRESIDENT|    5000.0|      5000|
|  null| SALESMAN|    1400.0|      1600|
|    10|     null|   2916.67|      5000|
|    10|    CLERK|    1300.0|      1300|
|    10|  MANAGER|    2450.0|      2450|
|    10|PRESIDENT|    5000.0|      5000|
|    20|     null|    2175.0|      3000|
|    20|  ANALYST|    3000.0|      3000|
|    20|    CLERK|     950.0|      1100|
|    20|  MANAGER|    2975.0|      2975|
|    30|     null|   1566.67|      2850|
|    30|    CLERK|     950.0|       950|
|    30|  MANAGER|    2850.0|      2850|
|    30| SALESMAN|    1400.0|      1600|
|    70|     null|    3200.0|      3200|
|    70|    CLERK|    3200.0|      3200|
+------+---------+----------+----------+



In [321]:
spark.stop()